In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI  
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.utilities import SQLDatabase
import os

# Load environment variables
google_api_key = "your  key"

# Initialize the SQL database
db = SQLDatabase.from_uri("sqlite:/// "+ "D:\Shantanu_Dev\Inboxone\database.db" )

# Initialize the language model
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, max_tokens=500, google_api_key=google_api_key)

# Create an SQL agent
agent_executor = create_sql_agent(llm, db=db, agent_type="zero-shot-react-description", verbose=True)

# Create a prompt template
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# Create the question-answer chain
question_answer_chain = create_stuff_documents_chain(llm, prompt)

# Example question
question = "tell me if any call scheduled on August 24"

# Execute the SQL agent to get the query result
query_result = agent_executor({"input": question})

# Initialize the embedding model
embeddings_model = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=google_api_key)


# Convert the query result to a list of texts (if needed)
texts = [query_result["output"]]  # Assuming the output is a single string, you can adjust as necessary

# Create a vector store from the query result (if needed)
# Create a vector store from the texts
vectorstore = FAISS.from_texts(texts, embeddings_model)

# Create a retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 10})

# Create the RAG chain
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Get the answer
response = rag_chain.invoke({"input": question})
print(response.get("answer", "No answer available."))
